In [1]:
import os
import sys
import pandas as pd
import torch
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
project_dir = os.path.dirname(notebook_dir)

if project_dir not in sys.path:
    sys.path.append(project_dir)
    
from machine_learning.preprocess_ml import DataPipeline
from machine_learning.train_models import TransformerFineTuner
    
# load data
df_a = pd.read_csv(r"C:\Users\johna\anaconda3\envs\twitter-analytics-env\twitter_issues_dashboard\twitter_issues_dashboard\data\02_intermediate\training_data.csv")
mapping = pd.read_csv(r"C:\Users\johna\anaconda3\envs\twitter-analytics-env\twitter_issues_dashboard\twitter_issues_dashboard\data\02_intermediate\topic_mapping.csv")

df = df_a.merge(right=mapping, how='inner', right_on='topic_sortedc_str', left_on='topic_sortedc_str')[['cleaned_text', 'topic_hl']]
df['topic_hl_encoded'] = df['topic_hl'].astype("category").cat.codes


# prepare data sets
pipeline = DataPipeline(df, target_col='topic_hl_encoded', text='cleaned_text', model='cardiffnlp/twitter-roberta-base')
train_loader = pipeline.get_train_data()
val_loader = pipeline.get_val_data()
test_loader = pipeline.get_test_data()

C:\Users\johna\anaconda3\envs\twitter-analytics-env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [41]:
from collections import Counter

# Define a dictionary to store the frequency of each class label
class_counts = Counter()

# Loop through the train_loader to count the frequency of each class label
for data in train_loader:
    # Get the class label for this data point
    label = data['labels']
    # Update the frequency count for this label
    class_counts[label] += 1

# Calculate the total number of data points in the dataset
total_count = sum(class_counts.values())

# Calculate the weight of each class label as the inverse of its frequency
class_weights = {label: total_count / (count * 6) for label, count in class_counts.items()}


In [47]:
import torch
from sklearn.utils.class_weight import compute_class_weight

# Assuming your dataset has 6 classes
num_classes = 6

# Assuming your training data is stored in a PyTorch DataLoader called "train_loader"
y_train = []
for batch in train_loader:
    y_train += batch["labels"].cpu().numpy().tolist()

class_weights = compute_class_weight('balanced', classes=range(num_classes), y=y_train)
class_weights = torch.tensor(class_weights, dtype=torch.float)


TypeError: 'int' object is not iterable

In [10]:
sample_data = train_loader[0]
print(sample_data.keys())


dict_keys(['input_ids', 'attention_mask', 'labels'])


In [51]:
class_counts = ndar[1240, 94, 74, 78, 114, 78]
total_samples = class_counts.sum()
class_weights = [total_samples / count for count in class_counts]
class_weights = class_weights / class_weights.max()


AttributeError: 'list' object has no attribute 'sum'

In [49]:
class_weights 

tensor([0.0597, 0.7872, 1.0000, 0.9487, 0.6491, 0.9487])

In [36]:
label = int(train_loader[700]['labels'])


In [37]:
label

0

In [52]:
from machine_learning.customer_trainer import CustomTrainer
